In [1]:
#importing libraries
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors

In [2]:
#source dataset path
path1 = r'D:\NSU\11th Semester\CSE445 - AZK\Project\Anime.csv'
path2 = r'D:\NSU\11th Semester\CSE445 - AZK\Project\profiles.csv'

In [3]:
#reading csv file
anime_data = pd.read_csv(path1)
anime_data.head()

,animeID,name,title_english,type,source,producers,genre,studio,episodes,score,scored_by,popularity,members,favorites,synopsis,premiered,related
0,1,Cowboy Bebop,Cowboy Bebop,TV,Original,['Bandai Visual'],"['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...",['Sunrise'],26.0,8.81,405664.0,39.0,795733.0,43460.0,"In the year 2071, humanity has colonized sever...",Spring 1998,"{'Adaptation': [{'mal_id': 173, 'type': 'manga..."
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,Movie,Original,"['Sunrise', 'Bandai Visual']","['Action', 'Drama', 'Mystery', 'Sci-Fi', 'Space']",['Bones'],1.0,8.41,120243.0,449.0,197791.0,776.0,"Another day, another bounty—such is the life o...",NaN,"{'Parent story': [{'mal_id': 1, 'type': 'anime..."
2,6,Trigun,Trigun,TV,Manga,['Victor Entertainment'],"['Action', 'Sci-Fi', 'Adventure', 'Comedy', 'D...",['Madhouse'],26.0,8.30,212537.0,146.0,408548.0,10432.0,"Vash the Stampede is the man with a $$60,000,0...",Spring 1998,"{'Adaptation': [{'mal_id': 703, 'type': 'manga..."
3,7,Witch Hunter Robin,Witch Hunter Robin,TV,Original,['Bandai Visual'],"['Action', 'Magic', 'Police', 'Supernatural', ...",['Sunrise'],26.0,7.33,32837.0,1171.0,79397.0,537.0,Witches are individuals with special powers li...,Summer 2002,{}
4,8,Bouken Ou Beet,Beet the Vandel Buster,TV,Manga,"['TV Tokyo', 'Dentsu']","['Adventure', 'Fantasy', 'Shounen', 'Supernatu...",['Toei Animation'],52.0,7.03,4894.0,3704.0,11708.0,14.0,It is the dark century and the people are suff...,Fall 2004,"{'Adaptation': [{'mal_id': 1348, 'type': 'mang..."


**Preparing Data

In [4]:
anime_data.columns

Index(['animeID', 'name', 'title_english', 'type', 'source', 'producers',
       'genre', 'studio', 'episodes', 'score', 'scored_by', 'popularity',
       'members', 'favorites', 'synopsis', 'premiered', 'related'],
      dtype='object')

In [5]:
#removing unnecessary Columns
del anime_data['producers']
del anime_data['studio']
del anime_data['popularity']
del anime_data['members']
del anime_data['favorites']
del anime_data['synopsis']
del anime_data['premiered']
del anime_data['related']

In [6]:
anime_data.head()

,animeID,name,title_english,type,source,genre,episodes,score,scored_by
0,1,Cowboy Bebop,Cowboy Bebop,TV,Original,"['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...",26.0,8.81,405664.0
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,Movie,Original,"['Action', 'Drama', 'Mystery', 'Sci-Fi', 'Space']",1.0,8.41,120243.0
2,6,Trigun,Trigun,TV,Manga,"['Action', 'Sci-Fi', 'Adventure', 'Comedy', 'D...",26.0,8.30,212537.0
3,7,Witch Hunter Robin,Witch Hunter Robin,TV,Original,"['Action', 'Magic', 'Police', 'Supernatural', ...",26.0,7.33,32837.0
4,8,Bouken Ou Beet,Beet the Vandel Buster,TV,Manga,"['Adventure', 'Fantasy', 'Shounen', 'Supernatu...",52.0,7.03,4894.0


In [7]:
#removing completely blank cell
anime_data = anime_data[anime_data.name != '404']

In [8]:
#removing [,] and ' from genre
temp1 = anime_data['genre']

temp1 = temp1.str.replace('[','',regex=True)
temp1 = temp1.str.replace(']','',regex=True)
temp1 = temp1.str.replace("'",'',regex=True)

#updating genre
anime_data['genre'] = temp1
anime_data.head()

,animeID,name,title_english,type,source,genre,episodes,score,scored_by
0,1,Cowboy Bebop,Cowboy Bebop,TV,Original,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",26.0,8.81,405664.0
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,Movie,Original,"Action, Drama, Mystery, Sci-Fi, Space",1.0,8.41,120243.0
2,6,Trigun,Trigun,TV,Manga,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",26.0,8.30,212537.0
3,7,Witch Hunter Robin,Witch Hunter Robin,TV,Original,"Action, Magic, Police, Supernatural, Drama, My...",26.0,7.33,32837.0
4,8,Bouken Ou Beet,Beet the Vandel Buster,TV,Manga,"Adventure, Fantasy, Shounen, Supernatural",52.0,7.03,4894.0


In [9]:
#checking if there is any unknown cell in type
anime_data[anime_data['type']=='Unknown'].head()

,animeID,name,title_english,type,source,genre,episodes,score,scored_by
8859,24023,Project758,NaN,Unknown,Unknown,Drama,NaN,NaN,0.0
9611,28613,Ginga Jinpuu Jinraiger,Galaxy Divine Wind Jinraiger,Unknown,Original,"Action, Adventure, Mecha",NaN,NaN,0.0
10339,30448,Mirai Arise,NaN,Unknown,Original,Sci-Fi,NaN,NaN,0.0
10344,30455,Kantai Collection: KanColle Zoku-hen,NaN,Unknown,Card game,"Action, Military, Sci-Fi, Slice of Life, School",NaN,NaN,0.0
11167,32455,Gekidol,NaN,Unknown,Unknown,Music,NaN,NaN,1.0


In [10]:
#removed unknown type animesource
anime_data = anime_data[anime_data.type != 'Unknown']

In [11]:
#checking if there is any unknown cell in source
anime_data[anime_data['source']=='Unknown'].head()

,animeID,name,title_english,type,source,genre,episodes,score,scored_by
60,88,Mobile Suit Gundam F91,Mobile Suit Gundam F91,Movie,Unknown,"Drama, Mecha, Military, Sci-Fi, Space",1.0,6.78,9833.0
78,108,Ou Dorobou Jing in Seventh Heaven,Jing: King of Bandits - Seventh Heaven,OVA,Unknown,"Adventure, Fantasy, Psychological, Comedy, Sho...",3.0,7.37,4665.0
140,176,Triangle Heart: Sweet Songs Forever,NaN,OVA,Unknown,"Adventure, Romance",4.0,6.24,2182.0
153,192,Love Hina Haru Special: Kimi Sakura Chiru Naka...,Love Hina Spring Movie,Special,Unknown,"Adventure, Comedy, Romance",1.0,7.27,27974.0
155,194,Macross Zero,Macross Zero,OVA,Unknown,"Adventure, Mecha, Military, Sci-Fi, Shounen",5.0,7.62,15264.0


In [12]:
#assuming unknow sources are original
anime_data.loc[anime_data['source']=='Unknown','source'] = 'Original'

In [13]:
#clearing cells with 0 scored by
anime_data.loc[anime_data['scored_by']==0,'scored_by'] = None
#anime_data[anime_data['scored_by']==0].head()

In [14]:
#checking all types for anime  
anime_data.type.unique()

array(['TV', 'Movie', 'OVA', 'Special', 'ONA', 'Music'], dtype=object)

In [15]:
#finding median of tv series episodes
tv = anime_data[anime_data['type'] == 'TV']
tv_median = tv['episodes'].median(skipna=True)
tv_median

24.0

In [18]:
#filling tv episodes null value with median
tv['episodes'].fillna(tv_median, inplace = True)

In [19]:
#finding average tv rating and average scored by
tv_rating_avg = tv['score'].mean(skipna=True)
tv_scoredBy_avg = math.floor(tv['scored_by'].mean(skipna=True))

#filling empty tv rating and scored by with average
tv['score'].fillna(tv_rating_avg, inplace = True)
tv['scored_by'].fillna(tv_scoredBy_avg, inplace = True)

In [20]:
#merging these values in main anime_data
anime_data.update(tv)

In [21]:
#not needed anymore
#replacing any unknown cell in episode with 1
#anime_data['episodes'].fillna(1, inplace = True)
#anime_data[anime_data['episodes']==1].head()


In [22]:
tv.head()

,animeID,name,title_english,type,source,genre,episodes,score,scored_by
0,1,Cowboy Bebop,Cowboy Bebop,TV,Original,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",26.0,8.81,405664.0
2,6,Trigun,Trigun,TV,Manga,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",26.0,8.30,212537.0
3,7,Witch Hunter Robin,Witch Hunter Robin,TV,Original,"Action, Magic, Police, Supernatural, Drama, My...",26.0,7.33,32837.0
4,8,Bouken Ou Beet,Beet the Vandel Buster,TV,Manga,"Adventure, Fantasy, Shounen, Supernatural",52.0,7.03,4894.0
5,16,Hachimitsu to Clover,Honey and Clover,TV,Manga,"Comedy, Drama, Josei, Romance, Slice of Life",24.0,8.12,57065.0


In [23]:
#Movies have 1 episode
movie = anime_data[anime_data['type'] == 'Movie']

#filling Movies episode null value with 1
movie['episodes'].fillna(1, inplace = True)

#finding average movie rating and average scored by
movie_rating_avg = movie['score'].mean(skipna=True)
movie_scoredBy_avg = math.floor(movie['scored_by'].mean(skipna=True))

#filling empty movie rating and scored by with average
movie['score'].fillna(movie_rating_avg, inplace = True)
movie['scored_by'].fillna(movie_scoredBy_avg, inplace = True)

#merging these values in main anime_data
anime_data.update(movie)

In [24]:
#finding median of OVA  episodes
ova = anime_data[anime_data['type'] == 'OVA']
ova_median = ova['episodes'].median(skipna=True)

#filling OVA episodes null value with median
ova['episodes'].fillna(ova_median, inplace = True)

#finding average OVA rating and average scored by
ova_rating_avg = ova['score'].mean(skipna=True)
ova_scoredBy_avg = math.floor(ova['scored_by'].mean(skipna=True))

#filling empty OVA rating and scored by with average
ova['score'].fillna(ova_rating_avg, inplace = True)
ova['scored_by'].fillna(ova_scoredBy_avg, inplace = True)

#merging these values in main anime_data
anime_data.update(ova)

In [25]:
#finding median of Special episodes
special = anime_data[anime_data['type'] == 'Special']
special_median = special['episodes'].median(skipna=True)

#filling Special episodes null value with median
special['episodes'].fillna(special_median, inplace = True)

#finding average Special rating and average scored by
special_rating_avg = special['score'].mean(skipna=True)
special_scoredBy_avg = math.floor(special['scored_by'].mean(skipna=True))

#filling empty Special rating and scored by with average
special['score'].fillna(special_rating_avg, inplace = True)
special['scored_by'].fillna(special_scoredBy_avg, inplace = True)

#merging these values in main anime_data
anime_data.update(special)

In [26]:
#finding median of ONA  episodes
ona = anime_data[anime_data['type'] == 'ONA']
ona_median = ona['episodes'].median(skipna=True)

#filling ONA null value with median
ona['episodes'].fillna(ona_median, inplace = True)

#finding average ONA rating and average scored by
ona_rating_avg = ona['score'].mean(skipna=True)
ona_scoredBy_avg = math.floor(ona['scored_by'].mean(skipna=True))

#filling empty ONA rating and scored by with average
ona['score'].fillna(ona_rating_avg, inplace = True)
ona['scored_by'].fillna(ona_scoredBy_avg, inplace = True)

#mergin these values in main anime_data
anime_data.update(ona)

In [27]:
#finding median of Music  episodes
music = anime_data[anime_data['type'] == 'Music']
music_median = ova['episodes'].median(skipna=True)

#filling Music null value with median
music['episodes'].fillna(music_median, inplace = True)

#finding average Music rating and average scored by
music_rating_avg = music['score'].mean(skipna=True)
music_scoredBy_avg = math.floor(music['scored_by'].mean(skipna=True))

#filling empty Music rating and scored by with average
music['score'].fillna(music_rating_avg, inplace = True)
music['scored_by'].fillna(music_scoredBy_avg, inplace = True)

#mergin these values in main anime_data
anime_data.update(music)

In [28]:
#checking if a Unknown values are replaced
episode = anime_data['episodes']
episode[15135]

2.0

In [29]:
#nan_rows = anime_data[anime_data.isnull().any(1)]
#nan_rows.head(50)
#anime_data.isnull().sum()

finding weighted rating

In [30]:
#finding mean rating across the whole anime_data
mean_animeRating = anime_data['score'].mean()

In [31]:
#funtion to find weighted reating
def W_R(a, C=mean_animeRating):
    v = a['scored_by']
    R = a['score']
    return (v/(v+1) * R) + (1/(1+v) * C)

In [32]:
#new column with weighted rating
anime_data['rating'] = anime_data.apply(W_R, axis=1)
#anime_data.head()

In [33]:
#anime_data.to_csv(r'D:\NSU\11th Semester\CSE445 - AZK\Project\test1.csv', index=False, mode= 'w')

In [34]:
anime_data.head()

,animeID,name,title_english,type,source,genre,episodes,score,scored_by,rating
0,1.0,Cowboy Bebop,Cowboy Bebop,TV,Original,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",26.0,8.81,405664.0,8.809994
1,5.0,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,Movie,Original,"Action, Drama, Mystery, Sci-Fi, Space",1.0,8.41,120243.0,8.409982
2,6.0,Trigun,Trigun,TV,Manga,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",26.0,8.30,212537.0,8.299991
3,7.0,Witch Hunter Robin,Witch Hunter Robin,TV,Original,"Action, Magic, Police, Supernatural, Drama, My...",26.0,7.33,32837.0,7.329968
4,8.0,Bouken Ou Beet,Beet the Vandel Buster,TV,Manga,"Adventure, Fantasy, Shounen, Supernatural",52.0,7.03,4894.0,7.029848


User Data file

In [35]:
#reading user data csv
user_data = pd.read_csv(path2)

In [36]:
user_data.head()

,profile,gender,birthday,favorites_anime,link
0,DesolatePsyche,Male,02-Oct-94,"['33352', '25013', '5530', '33674', '1482', '2...",https://myanimelist.net/profile/DesolatePsyche
1,baekbeans,Female,10-Nov-00,"['11061', '31964', '853', '20583', '918', '925...",https://myanimelist.net/profile/baekbeans
2,skrn,NaN,NaN,"['918', '2904', '11741', '17074', '23273', '32...",https://myanimelist.net/profile/skrn
3,edgewalker00,Male,Sep-05,"['5680', '849', '2904', '3588', '37349']",https://myanimelist.net/profile/edgewalker00
4,aManOfCulture99,Male,30-Oct-99,"['4181', '7791', '9617', '5680', '2167', '4382...",https://myanimelist.net/profile/aManOfCulture99


In [37]:
#replacing empty cells as unknown gender
user_data['gender'].fillna('unknown', inplace = True)
user_data.head()

,profile,gender,birthday,favorites_anime,link
0,DesolatePsyche,Male,02-Oct-94,"['33352', '25013', '5530', '33674', '1482', '2...",https://myanimelist.net/profile/DesolatePsyche
1,baekbeans,Female,10-Nov-00,"['11061', '31964', '853', '20583', '918', '925...",https://myanimelist.net/profile/baekbeans
2,skrn,unknown,NaN,"['918', '2904', '11741', '17074', '23273', '32...",https://myanimelist.net/profile/skrn
3,edgewalker00,Male,Sep-05,"['5680', '849', '2904', '3588', '37349']",https://myanimelist.net/profile/edgewalker00
4,aManOfCulture99,Male,30-Oct-99,"['4181', '7791', '9617', '5680', '2167', '4382...",https://myanimelist.net/profile/aManOfCulture99


In [38]:
#removed unknown type gender
user_data = user_data[user_data.gender != 'unknown']

In [39]:
len(user_data)

53856

In [40]:
#replacing [,] and ' from the favourites anime
temp = user_data['favorites_anime']

temp = temp.str.replace('[','',regex=True)
temp = temp.str.replace(']','',regex=True)
temp = temp.str.replace("'",'',regex=True)
temp.head()

0    33352, 25013, 5530, 33674, 1482, 269, 18245, 2...
1    11061, 31964, 853, 20583, 918, 9253, 34599, 35...
3                         5680, 849, 2904, 3588, 37349
4    4181, 7791, 9617, 5680, 2167, 4382, 849, 235, ...
7    11061, 30, 6594, 28701, 10087, 6746, 918, 1535...
Name: favorites_anime, dtype: object

In [41]:
#updating favourites anime
user_data['favorites_anime'] = temp
user_data.head()

,profile,gender,birthday,favorites_anime,link
0,DesolatePsyche,Male,02-Oct-94,"33352, 25013, 5530, 33674, 1482, 269, 18245, 2...",https://myanimelist.net/profile/DesolatePsyche
1,baekbeans,Female,10-Nov-00,"11061, 31964, 853, 20583, 918, 9253, 34599, 35...",https://myanimelist.net/profile/baekbeans
3,edgewalker00,Male,Sep-05,"5680, 849, 2904, 3588, 37349",https://myanimelist.net/profile/edgewalker00
4,aManOfCulture99,Male,30-Oct-99,"4181, 7791, 9617, 5680, 2167, 4382, 849, 235, ...",https://myanimelist.net/profile/aManOfCulture99
7,NIGGER_BONER,Male,01-Jan-85,"11061, 30, 6594, 28701, 10087, 6746, 918, 1535...",https://myanimelist.net/profile/NIGGER_BONER


In [42]:
#separating favourite anime genderwise
male_fav=user_data[user_data['gender']=='Male']
female_fav=user_data[user_data['gender']=='Female']
#male_fav.head()

In [43]:
#counting female fans of anime id wise
f_data = female_fav.favorites_anime.str.get_dummies(sep=', ').sum()

In [44]:
f_data

1        597
100       27
1000       6
10012     12
10015      4
        ... 
9981       3
9982       5
9989     333
9996       2
9999       2
Length: 2633, dtype: int64

In [45]:
#counting male fans of anime id wise
m_data = male_fav.favorites_anime.str.get_dummies(sep=', ').sum()

In [47]:
#creating a dataframe with anime id and female fan number
temp_1 = pd.DataFrame({'animeID': f_data.index, 'female_fan': f_data})
#df = pd.DataFrame({'female_fan': f_data})
temp_1.reset_index(drop=True, inplace=True)
temp_1.head()

,animeID,female_fan
0,1,597
1,100,27
2,1000,6
3,10012,12
4,10015,4


In [48]:
#creating another dataframe with anime id and male fan number
temp_2 = pd.DataFrame({'animeID': m_data.index, 'male_fan': m_data})
temp_2.reset_index(drop=True, inplace=True)
temp_2.head()

,animeID,male_fan
0,1,2922
1,100,1
2,1000,23
3,10012,67
4,10015,2


In [49]:
#merging two dataframe based on anime id
#temp_3 =  pd.DataFrame.merge(temp_2,temp_1,on='animeID')
temp_3 = temp_2.merge(temp_1,how='left', left_on='animeID', right_on='animeID')
temp_3.head()

,animeID,male_fan,female_fan
0,1,2922,597.0
1,100,1,27.0
2,1000,23,6.0
3,10012,67,12.0
4,10015,2,4.0


In [50]:
#temp_3.to_csv(r'D:\NSU\11th Semester\CSE445 - AZK\Project\user_data.csv')

In [51]:
#converting anime id from str to numeric
temp_3["animeID"] = pd.to_numeric(temp_3["animeID"])

In [52]:
#creating a new dataframe combining anime data and user data
#df =  pd.DataFrame.merge(anime_data,temp_3,on='animeID')
df = anime_data.merge(temp_3,how='left', left_on='animeID', right_on='animeID')
df.head()

,animeID,name,title_english,type,source,genre,episodes,score,scored_by,rating,male_fan,female_fan
0,1.0,Cowboy Bebop,Cowboy Bebop,TV,Original,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",26.0,8.81,405664.0,8.809994,2922.0,597.0
1,5.0,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,Movie,Original,"Action, Drama, Mystery, Sci-Fi, Space",1.0,8.41,120243.0,8.409982,36.0,11.0
2,6.0,Trigun,Trigun,TV,Manga,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",26.0,8.30,212537.0,8.299991,609.0,179.0
3,7.0,Witch Hunter Robin,Witch Hunter Robin,TV,Original,"Action, Magic, Police, Supernatural, Drama, My...",26.0,7.33,32837.0,7.329968,18.0,24.0
4,8.0,Bouken Ou Beet,Beet the Vandel Buster,TV,Manga,"Adventure, Fantasy, Shounen, Supernatural",52.0,7.03,4894.0,7.029848,NaN,NaN


In [53]:
#filling empty cells with 0
df['male_fan'].fillna(0, inplace=True)
df['female_fan'].fillna(0, inplace=True)

In [54]:
#funtion to calculate fanbase gender biasness
def Fan(a):
    x = a['male_fan']
    y = a['female_fan']
    return (x-y)

In [55]:
#new column with fan base
df['fan_base'] = df.apply(Fan, axis=1)

In [56]:
df.head()

,animeID,name,title_english,type,source,genre,episodes,score,scored_by,rating,male_fan,female_fan,fan_base
0,1.0,Cowboy Bebop,Cowboy Bebop,TV,Original,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",26.0,8.81,405664.0,8.809994,2922.0,597.0,2325.0
1,5.0,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,Movie,Original,"Action, Drama, Mystery, Sci-Fi, Space",1.0,8.41,120243.0,8.409982,36.0,11.0,25.0
2,6.0,Trigun,Trigun,TV,Manga,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",26.0,8.30,212537.0,8.299991,609.0,179.0,430.0
3,7.0,Witch Hunter Robin,Witch Hunter Robin,TV,Original,"Action, Magic, Police, Supernatural, Drama, My...",26.0,7.33,32837.0,7.329968,18.0,24.0,-6.0
4,8.0,Bouken Ou Beet,Beet the Vandel Buster,TV,Manga,"Adventure, Fantasy, Shounen, Supernatural",52.0,7.03,4894.0,7.029848,0.0,0.0,0.0


In [57]:
#saving dataframe to csv file
df.to_csv(r'D:\NSU\11th Semester\CSE445 - AZK\Project\anime_data.csv')

In [58]:
#converting feature to categorical values
temp_type = pd.get_dummies(df['type'])
temp_source = pd.get_dummies(df['source'])
temp_genre = df['genre'].str.get_dummies(sep=', ')

#temp_type.head()

In [59]:
#taking the features into a dataframe
anime = pd.concat([temp_type,temp_source,temp_genre,df['episodes'],df['rating'],df['fan_base']], axis=1)

In [60]:
anime.head()

,Movie,Music,ONA,OVA,Special,TV,4-koma manga,Book,Card game,Digital manga,...,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri,episodes,rating,fan_base
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,26.0,8.809994,2325.0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,8.409982,25.0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,26.0,8.299991,430.0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,26.0,7.329968,-6.0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,52.0,7.029848,0.0


In [61]:
#Scaling the values
scaler = MinMaxScaler()
anime = scaler.fit_transform(anime)
np.round(anime,2)

array([[0.  , 0.  , 0.  , ..., 0.01, 0.94, 0.58],
       [1.  , 0.  , 0.  , ..., 0.  , 0.89, 0.18],
       [0.  , 0.  , 0.  , ..., 0.01, 0.87, 0.25],
       ...,
       [0.  , 0.  , 1.  , ..., 0.  , 0.52, 0.17],
       [0.  , 0.  , 0.  , ..., 0.01, 0.66, 0.18],
       [0.  , 0.  , 0.  , ..., 0.01, 0.66, 0.17]])

In [68]:
#clustering using KNN
nbrs = NearestNeighbors(n_neighbors=11, algorithm='ball_tree').fit(anime)
distances, indices = nbrs.kneighbors(anime)

In [83]:
#funtion to get recommendation
def get_recommendation(name):
    rec_id=df.index[df['name']==name].tolist()[0]
    recommend = pd.DataFrame(columns = ['Recommended Anime', 'English Title', 'Type', 'Rating', 'Genre'])
    for id in indices[rec_id][1:]:
        new_row = {'Recommended Anime':df.iloc[id]['name'], 'English Title':df.iloc[id]['title_english'], 'Type':df.iloc[id]['type'], 'Rating':np.round(df.iloc[id]['rating'],2), 'Genre':df.iloc[id]['genre'] }
        recommend = recommend.append(new_row, ignore_index=True)
    print("Anime Recommendation for:", name, '/',df.iloc[rec_id]['title_english'])
    display(recommend)

In [90]:
get_recommendation('Cowboy Bebop')

Anime Recommendation for: Cowboy Bebop / Cowboy Bebop


,Recommended Anime,English Title,Type,Rating,Genre
0,Ginga Tetsudou Monogatari,The Galaxy Railways,TV,7.25,"Action, Adventure, Space, Drama, Sci-Fi"
1,Ginga Tetsudou Monogatari: Eien e no Bunkiten,NaN,TV,6.98,"Action, Adventure, Space, Drama, Sci-Fi"
2,Seihou Tenshi Angel Links,Angel Links,TV,6.02,"Action, Adventure, Space, Comedy, Romance, Dra..."
3,Uchuu Patrol Luluco,Space Patrol Luluco,TV,7.68,"Action, Adventure, Comedy, Space"
4,Uchuu Senkan Yamato,Star Blazers: The Quest for Iscandar,TV,7.67,"Action, Adventure, Drama, Military, Sci-Fi, Space"
5,Uchuu Senkan Yamato 2,Star Blazers: The Comet Empire,TV,7.55,"Action, Military, Sci-Fi, Adventure, Space, Drama"
6,Cowboy Bebop: Yose Atsume Blues,Cowboy Bebop Session XX,Special,7.49,"Action, Adventure, Comedy, Drama, Sci-Fi, Space"
7,Uchuu Senkan Yamato III,Star Blazers: The Bolar Wars,TV,7.21,"Action, Military, Sci-Fi, Adventure, Space, Drama"
8,Uchuu Densetsu Ulysses 31,Ulysses 31,TV,7.15,"Action, Adventure, Sci-Fi, Space"
9,Iron Kid,Eon Kid,TV,7.06,"Action, Adventure, Comedy, Sci-Fi"
